## ANÁLISE INSUMO-PRODUTO: O PAPEL DO SETOR DE CELULOSE E PAPEL NA ECONOMIA BRASILEIRA

### Importar pacotes

In [ ]:
import numpy as np
import pandas as pd

### Ler as matrizes

In [ ]:
nAnoMip = 2018
nSetores = 68
nProdutos = 128

nColunaOferta = 7

nColunaMComercio = 1
nColunaMTransporte = 2
nColunaII = 3 
nColunaIPI = 4 
nColunaICMS = 5 
nColunaOI = 6

In [ ]:
def LerMatriz(sArquivo, sNomePasta, nLinhaInicial, nColunaInicial, nLinhas, nColunas):
    dSheet = pd.read_excel(sArquivo, sNomePasta, header=None, index_col=None)
    mMatriz = dSheet.values[nLinhaInicial:nLinhaInicial+nLinhas, nColunaInicial:nColunaInicial+nColunas]
    return mMatriz

sArquivoTabelaRecursos = str(nSetores)+'_tab1_'+str(nAnoMip)+'.xls'
sArquivoTabelaUsos = str(nSetores)+'_tab2_'+str(nAnoMip)+'.xls'

mProducao = LerMatriz(sArquivoTabelaRecursos, 'producao', 5, 2, nProdutos, nSetores)
mOferta = LerMatriz(sArquivoTabelaRecursos, 'oferta', 5, 2, nProdutos, nColunaOferta)
mImportacao = LerMatriz(sArquivoTabelaRecursos, 'importacao', 5, 2, nProdutos, 1)

mCI = LerMatriz(sArquivoTabelaUsos, 'CI', 5, 2, nProdutos, nSetores)
mDemanda = LerMatriz(sArquivoTabelaUsos, 'demanda', 5, 2, nProdutos, 6)
mVA = LerMatriz(sArquivoTabelaUsos, 'VA', 5, 1, 14, nSetores)

### Estimar Tabela de Usos a Preços Básicos

#### Tirar a coluna Estoque

In [ ]:
mDemandaFinalSemEstoque = np.copy(mDemanda)
mDemandaFinalSemEstoque[:,5] = 0

#### Concatenar Consumo Intermediário com Demanda Final

In [ ]:
mConsumoTotalSemEstoque = np.concatenate((mCI, mDemandaFinalSemEstoque), axis=1)

#### Estimar a matriz de distribuição do Consumo Total

In [ ]:
def MatrizDistribuicao(mMatrizEntrada):
    vTotalProduto = np.sum(mMatrizEntrada, axis=1, keepdims=True)
    mMatrizSaida = mMatrizEntrada / vTotalProduto
    return mMatrizSaida

mMatrizDistribuicao = MatrizDistribuicao(mConsumoTotalSemEstoque)

#### Multiplicar pelas margens, impostos e importações

##### Margem de comércio

In [ ]:
# Primeiro pegamos a proporção das margens na coluna
vMargemC = mOferta[:,nColunaMComercio]
vMargemC = vMargemC.reshape((nProdutos, 1))
vPropMargemC = vMargemC[92:94] / np.sum(vMargemC[92:94])

# Em seguida calculamos a matriz de distribuição com os valores cheios
mMatrizSaidaC = mMatrizDistribuicao * vMargemC

# Em seguida somamos os valores pelas linhas da m. distribuição desconsiderando as linhas das margens negativas
mMargemDistribuidaC = np.sum(mMatrizSaidaC[:92], axis=0) + np.sum(mMatrizSaidaC[94:], axis=0)

# Em seguida definimos os valores das linhas do vetor como sendo a proporsão
mMatrizSaidaC[92,:] = mMargemDistribuidaC * vPropMargemC[0,None] * (-1)
mMatrizSaidaC[93,:] = mMargemDistribuidaC * vPropMargemC[1,None] * (-1)

##### Margem de transporte

In [ ]:
# Primeiro pegamos a proporção das margens na coluna
vMargemT = mOferta[:,nColunaMTransporte]
vMargemT = vMargemT.reshape((nProdutos, 1))
vPropMargemT = vMargemT[94:98] / np.sum(vMargemT[94:98])

# Em seguida calculamos a matriz de distribuição com os valores cheios
mMatrizSaidaT = mMatrizDistribuicao * vMargemT

# Em seguida somamos os valores pelas linhas da m. distribuição desconsiderando as linhas das margens negativas
mMargemDistribuidaT = np.sum(mMatrizSaidaT[:94], axis=0) + np.sum(mMatrizSaidaT[98:], axis=0)

# Em seguida definimos os valores das linhas do vetor como sendo a proporsão
mMatrizSaidaT[94,:] = mMargemDistribuidaT * vPropMargemT[0,None] * (-1)
mMatrizSaidaT[95,:] = mMargemDistribuidaT * vPropMargemT[1,None] * (-1)
mMatrizSaidaT[96,:] = mMargemDistribuidaT * vPropMargemT[2,None] * (-1)
mMatrizSaidaT[97,:] = mMargemDistribuidaT * vPropMargemT[3,None] * (-1)

##### Demais Margens

In [ ]:
mMargemIPI = mMatrizDistribuicao * mOferta[:, nColunaIPI].reshape((nProdutos, 1))
mMargemICMS = mMatrizDistribuicao * mOferta[:, nColunaICMS].reshape((nProdutos, 1))
mMargemOI = mMatrizDistribuicao * mOferta[:, nColunaOI].reshape((nProdutos, 1))

#### Tirar exportações

In [ ]:
mConsumoTotalSemExportacao = np.copy(mConsumoTotalSemEstoque)
mConsumoTotalSemExportacao[:,68] = 0

#### Matriz distribuição sem exportações e sem estoque

In [ ]:
mDistribuicaoSemExportacao = MatrizDistribuicao(mConsumoTotalSemExportacao)

##### Margem II e Importações

In [ ]:
mMargemII = mDistribuicaoSemExportacao * mOferta[:, nColunaII].reshape((nProdutos, 1))
mMargemImportacao = mDistribuicaoSemExportacao * mImportacao[:, 0].reshape((nProdutos, 1))

### Oferta Global a Preços de Mercado

In [ ]:
mConsumoTotal = np.concatenate((mCI, mDemanda), axis=1)

### Oferta Nacional a Preços Básicos

In [ ]:
mNacionalBasico = mConsumoTotal - mMatrizSaidaC - mMargemICMS - mMargemII - mMargemImportacao - mMargemIPI - mMargemOI - mMatrizSaidaT

### Estimação MIP

In [ ]:
mU = mNacionalBasico[:,0:nSetores]
mE = mNacionalBasico[:,nSetores:]

mX = mVA[12,:]
mXChapeu = np.diagflat(1/mX)
mB = np.dot(mU, mXChapeu)

mProducaoV = np.copy(mProducao)
mV = mProducao.T

vQ = np.sum(mV, axis=0)
mQChapeu = np.diagflat(1/vQ)

mD = np.dot(mV, mQChapeu)
mA = np.dot(mD, mB).astype('float')
mY = np.dot(mD, mE).astype('float')
# Construção do CI setor x setor
mZ = np.dot(mD, mU).astype('float')
mI = np.eye(nSetores)
mLeontief = np.linalg.inv(mI - mA)

### Construção da MIP

In [ ]:
# Linha somatório do Consumo Intermediário
vZTotalO = np.sum(mZ, axis=0, keepdims=True)

# Linha somatório da Importação
vMTotal = np.sum(mMargemImportacao, axis=0, keepdims=True)
vMCI = vMTotal[:, :nSetores]
vMD = vMTotal[:, nSetores:]

# Linha somatório dos tributos
mTtotal = mMargemICMS + mMargemII + mMargemIPI + mMargemOI
vTtotal = np.sum(mTtotal, axis=0, keepdims=True)
vTCI = vTtotal[:, :nSetores]
vTD = vTtotal[:, nSetores:]

In [ ]:
# Concatenar
mOfertaCI = np.concatenate((mZ, vZTotalO, vMCI, vTCI, mVA), axis=0)

# Coluna somatório
vZTotalD = np.sum(mOfertaCI, axis=1, keepdims=True)

# Linha somatório
vTotalLinhaDN = np.sum(mY, axis=0, keepdims=True)

# Concatenar
mDemandaFinal = np.concatenate((mY, vTotalLinhaDN, vMD, vTD), axis=0)

# Linha somatório
vTotalLinhaDF = np.sum(mDemandaFinal, axis=0, keepdims=True)

# Concatenar
mDemanda = np.concatenate((mDemandaFinal, vTotalLinhaDF, np.zeros([13,6], dtype=float)), axis=0)

In [ ]:
vTotalDemandaFinal = np.sum(mDemanda, axis=1, keepdims=True)

vDemandaTotal = vZTotalD + vTotalDemandaFinal

mMIP = np.concatenate((mOfertaCI, vZTotalD, mDemanda, vTotalDemandaFinal, vDemandaTotal), axis=1)

#### Checagem de equilíbrio

##### Oferta Total - Demanda Total

In [ ]:
np.sum(np.sum(mZ, axis=1) + np.sum(mY, axis=1)) - np.sum(vZTotalO + vMCI + vTCI + mVA[0])

0.0

In [ ]:
np.sum(np.sum(mZ, axis=1) + np.sum(mY, axis=1)) - np.sum(mX)

0.0

#### Exportar matrizes

##### MIP

In [ ]:
df = pd.DataFrame(mMIP)
df.to_excel('mMIP'+str(nAnoMip)+'.xlsx', index=False)

##### Matriz Leontief

In [ ]:
df = pd.DataFrame(mLeontief)
df.to_excel('mLeontief'+str(nAnoMip)+'.xlsx', index=False)

##### Matriz A

In [ ]:
df = pd.DataFrame(mA)
df.to_excel('mA'+str(nAnoMip)+'.xlsx', index=False)

### Modelo Fechado

In [ ]:
vConsumoFamilias = mY[:,3].reshape((nSetores,1))

vAux = (mVA[1,:] + mVA[8,:]).reshape((1,nSetores))

In [ ]:
vDifEOB = ((mVA[9,:] / np.sum(mVA[9,:])) * (np.sum(vConsumoFamilias) - np.sum(vAux))).reshape((1, nSetores))

In [ ]:
vRendaFamilias = vAux + vDifEOB

#### Checagem de equilíbrio

In [ ]:
np.sum(vRendaFamilias) - np.sum(vConsumoFamilias)

# O resultado deu muito próximo de zero

-9.313225746154785e-10

#### Construção do Modelo Fechado

In [ ]:
mZbarra = np.concatenate((mZ, vRendaFamilias), axis=0)

nAux = np.sum(vConsumoFamilias).reshape((1,1))
vAux = np.concatenate((vConsumoFamilias, nAux), axis=0)

mZbarra = np.concatenate((mZbarra, vAux), axis=1)

vVBP = mVA[12].reshape((1,nSetores))
nAuxvbp = np.array(np.sum(vVBP)).reshape((1,1))
vVBPbarra = np.concatenate((vVBP, nAuxvbp), axis=1)

mAbarra = np.zeros([nSetores+1, nSetores+1])
mAbarra[:,:] = mZbarra[:,:] / vVBPbarra[0,:]
mIbarra = np.eye(nSetores+1)
mLeontiefbarra = np.linalg.inv(mIbarra - mAbarra)
mLeontiefbarra.shape

(69, 69)

#### Exportando a Matriz de Leontief Modelo Fechado

In [ ]:
dfLeontiefbarradf = pd.DataFrame(mLeontiefbarra)
dfLeontiefbarradf.to_excel('mLeontiefFechado'+str(nAnoMip)+'.xlsx', index=False)

### Multiplicadores e Geradores

#### Nome setores

In [ ]:
vSetores = pd.read_excel('68_tab1_2018.xls', 'producao', header=None, index_col=None)
vSetores = vSetores.values[3,2:70]
for i in range(68):
    vSetores[i] = vSetores[i][5:]
vSetores

array(['Agricultura, inclusive o apoio à agricultura e a pós-colheita',
       'Pecuária, inclusive o apoio à pecuária',
       'Produção florestal; pesca e aquicultura',
       'Extração de carvão mineral e de minerais não-metálicos',
       'Extração de petróleo e gás, inclusive as atividades de apoio',
       'Extração de minério de ferro, inclusive beneficiamentos e a aglomeração',
       'Extração de minerais metálicos não-ferrosos, inclusive beneficiamentos',
       'Abate e produtos de carne, inclusive os produtos do laticínio e da pesca',
       'Fabricação e refino de açúcar', 'Outros produtos alimentares',
       'Fabricação de bebidas', 'Fabricação de produtos do fumo',
       'Fabricação de produtos têxteis',
       'Confecção de artefatos do vestuário e acessórios',
       'Fabricação de calçados e de artefatos de couro',
       'Fabricação de produtos da madeira',
       'Fabricação de celulose, papel e produtos de papel',
       'Impressão e reprodução de gravações',
   

##### Produção

In [ ]:
vMultSimplesProd = np.sum(mLeontief, axis=0, keepdims=True)
vMultTotalProd = np.sum(mLeontiefbarra, axis=0, keepdims=True)
vEfInduzido = vMultTotalProd[0,:68] - vMultSimplesProd
vEfDireto = np.sum(mA, axis=0, keepdims=True)
vEfIndireto = vMultSimplesProd - vEfDireto

In [ ]:
mMultiplicadores = {
    'Setores': [],
    'Total': [],
    'Simples': [],
    'Induzido': [],
    'Direto': [],
    'Indireto': []
}

for i in range(68):
    mMultiplicadores['Setores'].append(vSetores[i])
    mMultiplicadores['Total'].append(vMultTotalProd[:,i][0])
    mMultiplicadores['Simples'].append(vMultSimplesProd[:,i][0])
    mMultiplicadores['Induzido'].append(vEfInduzido[:,i][0])
    mMultiplicadores['Direto'].append(vEfDireto[:,i][0])
    mMultiplicadores['Indireto'].append(vEfIndireto[:,i][0])

In [ ]:
# Aqui, utilizou-se a MIP para facilitar os cálculos.

def FuncaoCalculoMultiplicadores(nomeoperacao, linhaoperacao, linhavbp, nsetores):
    vRequisito = mMIP[linhaoperacao,:][0:nsetores] / mMIP[linhavbp,:][0:nsetores]
    mRequisito = np.diagflat(vRequisito)

    mGerador = np.dot(mRequisito, mLeontief)
    vMultSimples = np.sum(mGerador, axis=0, keepdims=True)

    mGeradorFechado = np.dot(mRequisito, mLeontiefbarra[0:nsetores,0:nsetores])
    vMultTotal = np.sum(mGeradorFechado, axis=0, keepdims=True)

    vEfInduzido = vMultTotal - vMultSimples
    vEfeitoIndireto = vMultSimples - vRequisito
    
    mTabelaMultiplicadores = {
    'Setores': [],
    'Total': [],
    'Simples': [],
    'Induzido': [],
    'Direto': [],
    'Indireto': []
    }
    
    for i in range(68):
        mTabelaMultiplicadores['Setores'].append(vSetores[i])
        mTabelaMultiplicadores['Total'].append(vMultTotal[:,i][0])
        mTabelaMultiplicadores['Simples'].append(vMultSimples[:,i][0])
        mTabelaMultiplicadores['Induzido'].append(vEfInduzido[:,i][0])
        mTabelaMultiplicadores['Direto'].append(vRequisito[i])
        mTabelaMultiplicadores['Indireto'].append(vEfeitoIndireto[:,i][0])        

    return mTabelaMultiplicadores

##### Emprego

In [ ]:
TabelaMultiplicadoresEmprego = FuncaoCalculoMultiplicadores('Emprego', 84, 83, 68)

##### Renda

In [ ]:
TabelaMultiplicadoresRenda = FuncaoCalculoMultiplicadores('Renda', 72, 83, 68)

##### PIB

In [ ]:
TabelaMultiplicadoresVA = FuncaoCalculoMultiplicadores('VA', 71, 83, 68)

##### Impostos

In [ ]:
TabelaMultiplicadoresTributos = FuncaoCalculoMultiplicadores('Tributos', 70, 83, 68)

### Índices de Ligação

##### Média linhas e colunas

In [ ]:
# axis=0: média da coluna. axis=1: média da linha
vMediacolunasB = np.mean(mLeontief, axis=0)
vMedialinhasB = np.mean(mLeontief, axis=1)
nMediatodosB = np.mean(mLeontief)

##### Normalização

In [ ]:
# Uj
vIndiceTras = vMediacolunasB / nMediatodosB
# Ui
vIndiceFrente = vMedialinhasB / nMediatodosB

##### Coeficientes de variação

In [ ]:
# Quanto menor o coeficiente, mais uniforme é o seu impacto
vVJ = np.std(mLeontief, axis=0, ddof=1) / vMediacolunasB
vVI = np.std(mLeontief, axis=1, ddof=1) / vMedialinhasB

### Modelo de Preços

In [ ]:
d = mMIP[69:72,:68]
d = d / mMIP[83,:68]

In [ ]:
p = np.dot(mLeontief.T, d.T)
p.shape

(68, 3)

#### Checagem

In [ ]:
np.sum(p, axis=1)

# Valores muito próximos de 1

array([0.9999999999999998, 1.0000000000000004, 0.9999999999999993,
       1.0000000000000004, 1.0000000000000002, 0.9999999999999998, 1.0,
       1.0000000000000002, 0.9999999999999999, 0.9999999999999997,
       0.9999999999999998, 1.0, 1.0000000000000009, 0.9999999999999999,
       1.0000000000000002, 1.0000000000000007, 1.0000000000000004, 1.0,
       1.0, 1.0000000000000004, 0.9999999999999997, 0.9999999999999996,
       0.9999999999999998, 1.0000000000000002, 1.0000000000000004, 1.0,
       1.0000000000000009, 1.0000000000000002, 0.9999999999999998,
       0.9999999999999997, 1.0000000000000002, 1.0000000000000004, 1.0,
       1.0, 0.9999999999999999, 1.0000000000000002, 0.9999999999999998,
       1.0000000000000004, 0.9999999999999999, 0.9999999999999999,
       1.0000000000000007, 1.0000000000000004, 1.0000000000000002,
       1.0000000000000004, 0.9999999999999999, 0.9999999999999997,
       1.0000000000000002, 0.9999999999999996, 1.0, 1.0000000000000004,
       0.9999999999999

### Índices Puros de Ligação

In [ ]:
mIPL = np.zeros([nSetores, 3])
mYtotal = np.sum(mY, axis=1, keepdims=True)

In [ ]:
for n in range(nSetores):
    i = n
    j = i
    vRestoEconomia = [True] * nSetores
    vRestoEconomia[i] = False
    
    Yj = mYtotal[i]
    Yr = mYtotal[vRestoEconomia]
    Ajj = mA[i, j]
    Deltaj = (1/(1 - Ajj))
    Ajr = mA[i, vRestoEconomia].reshape((1,nSetores-1))
    Arj = mA[vRestoEconomia, i].reshape((nSetores-1,1))
    Arr = mA[np.ix_(vRestoEconomia, vRestoEconomia)]
    mI = np.eye(nSetores - 1)
    Deltar = np.linalg.inv(mI - Arr)
    
    PFL = np.dot(Deltaj,Ajr).dot(Deltar).dot(Yr)
    PBL = np.sum(Deltar.dot(Arj).dot(Deltaj).dot(Yj))
    PTL = PBL + PFL
    
    mIPL[n,0] = PBL
    mIPL[n,1] = PFL
    mIPL[n,2] = PTL
    
mIPLn = (np.sum(mIPL, axis=0) / nSetores)
mIPLn = np.diagflat(1 / mIPLn)
mIPLN = np.dot(mIPL, mIPLn) 

### Apresentação de resultados

##### Indicadores frente e trás

In [ ]:
df_ind_tras_frente = {
    'Setores': vSetores,
    'BL': vMediacolunasB,
    'PD': vIndiceTras,
    'FL': vMedialinhasB,
    'SD': vIndiceFrente
}

In [ ]:
df_ind_tras_frente = pd.DataFrame(df_ind_tras_frente)

In [ ]:
df_ind_tras_frente['Ranking BL'] = df_ind_tras_frente['PD'].rank(ascending=False)
df_ind_tras_frente['Ranking FL'] = df_ind_tras_frente['SD'].rank(ascending=False)

In [ ]:
df_ind_tras_frente = df_ind_tras_frente[['Setores', 'BL', 'PD', 'Ranking BL', 'FL', 'SD', 'Ranking FL']]

df_ind_tras_frente[df_ind_tras_frente['Setores'] == 'Fabricação de celulose, papel e produtos de papel'].reset_index(drop=True)

,Setores,BL,PD,Ranking BL,FL,SD,Ranking FL
0,"Fabricação de celulose, papel e produtos de papel",0.030472,1.140986,13.0,0.028959,1.084361,17.0


##### Poder e sensibilidade de dispersão

In [ ]:
df_ind_pd_sd = {
    'Setores': vSetores,
    'CV PD': vVJ,
    'CV SD': vVI
}

df_ind_pd_sd = pd.DataFrame(df_ind_pd_sd)

df_ind_pd_sd['Ranking PD'] = df_ind_pd_sd['CV PD'].rank(ascending=False)
df_ind_pd_sd['Ranking SD'] = df_ind_pd_sd['CV SD'].rank(ascending=False)

df_ind_pd_sd = df_ind_pd_sd[['Setores', 'CV PD', 'Ranking PD', 'CV SD', 'Ranking SD']]
df_ind_pd_sd[df_ind_pd_sd['Setores'] == 'Fabricação de celulose, papel e produtos de papel'].reset_index(drop=True)

,Setores,CV PD,Ranking PD,CV SD,Ranking SD
0,"Fabricação de celulose, papel e produtos de papel",4.754894,40.0,4.977179,48.0


##### Setores chave

In [ ]:
df_chaves = {
    'Setores': [],
    'BL': [],
    'PD': [],
    'FL': [],
    'SD': []
}

In [ ]:
for i in range(68):
    if (vIndiceTras[i] > 1) & (vIndiceFrente[i] > 1):
        df_chaves['Setores'].append(vSetores[i])
        df_chaves['BL'].append(vMediacolunasB[i])
        df_chaves['PD'].append(vIndiceTras[i])
        df_chaves['FL'].append(vMedialinhasB[i])
        df_chaves['SD'].append(vIndiceFrente[i])

In [ ]:
df_chaves = pd.DataFrame(df_chaves)
df_chaves

,Setores,BL,PD,FL,SD
0,Outros produtos alimentares,0.034014,1.273646,0.026873,1.006244
1,"Fabricação de celulose, papel e produtos de papel",0.030472,1.140986,0.028959,1.084361
2,Refino de petróleo e coquerias,0.034221,1.281389,0.071247,2.667784
3,Fabricação de químicos orgânicos e inorgânicos...,0.029467,1.103388,0.050681,1.897711
4,"Fabricação de defensivos, desinfestantes, tint...",0.030344,1.136216,0.029757,1.114213
5,Fabricação de produtos de borracha e de materi...,0.030245,1.132509,0.031825,1.191654
6,"Produção de ferro-gusa/ferroligas, siderurgia ...",0.031093,1.164264,0.033250,1.245007
7,Metalurgia de metais não-ferrosos e a fundição...,0.031241,1.169781,0.027154,1.016753
8,"Fabricação de produtos de metal, exceto máquin...",0.030360,1.136807,0.028009,1.048758
9,"Manutenção, reparação e instalação de máquinas...",0.026716,1.000367,0.031860,1.192985


##### Índices puros de ligação

In [ ]:
df_indices_puros = {
    'Setores': vSetores,
    'PBL': mIPLN[:,0],
    'PFL': mIPLN[:,1],
    'PTL': mIPLN[:,2]
}

In [ ]:
df_indices_puros = pd.DataFrame(df_indices_puros)

df_indices_puros['Ranking PBL'] = df_indices_puros['PBL'].rank(ascending=False)
df_indices_puros['Ranking PFL'] = df_indices_puros['PFL'].rank(ascending=False)
df_indices_puros['Ranking PTL'] = df_indices_puros['PTL'].rank(ascending=False)

df_indices_puros = df_indices_puros[['Setores', 'PBL', 'Ranking PBL', 'PFL', 'Ranking PFL', 'PTL', 'Ranking PTL']]
df_indices_puros.sort_values(by='PTL', ascending=False)

,Setores,PBL,Ranking PBL,PFL,Ranking PFL,PTL,Ranking PTL
41,"Comércio por atacado e a varejo, exceto veícul...",5.409040,1.0,6.734391,1.0,6.070550,1.0
39,Construção,5.369328,2.0,0.881289,25.0,3.129255,2.0
52,"Intermediação financeira, seguros e previdênci...",1.436476,14.0,4.662280,2.0,3.046541,3.0
18,Refino de petróleo e coquerias,1.864889,10.0,4.045099,3.0,2.953077,4.0
7,"Abate e produtos de carne, inclusive os produt...",5.129641,3.0,0.548978,35.0,2.843338,5.0
...,...,...,...,...,...,...,...
3,Extração de carvão mineral e de minerais não-m...,0.018636,65.0,0.289973,47.0,0.154066,64.0
11,Fabricação de produtos do fumo,0.267128,48.0,0.001162,67.0,0.134379,65.0
6,"Extração de minerais metálicos não-ferrosos, i...",0.130491,58.0,0.126378,58.0,0.128438,66.0
48,Edição e edição integrada à impressão,0.103407,60.0,0.137398,57.0,0.120373,67.0


In [ ]:
df_indices_puros[df_indices_puros['Setores'] == 'Fabricação de celulose, papel e produtos de papel'].reset_index(drop=True)

,Setores,PBL,Ranking PBL,PFL,Ranking PFL,PTL,Ranking PTL
0,"Fabricação de celulose, papel e produtos de papel",0.659603,31.0,0.801274,27.0,0.730314,32.0


##### Formação preços

In [ ]:
df_precos = {
    'Setores': vSetores,
    '% Importação': p[:, 0],
    '% Impostos': p[:, 1],
    '% Valor Adicionado': p[:,2]
}

df_precos = pd.DataFrame(df_precos)
df_precos[df_precos['Setores'] == 'Fabricação de celulose, papel e produtos de papel'].reset_index(drop=True)

,Setores,% Importação,% Impostos,% Valor Adicionado
0,"Fabricação de celulose, papel e produtos de papel",0.170019,0.0942676,0.735713


In [ ]:
df_precos['% Importação'].mean()

0.13544402251911747

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df_precos.sort_values(by='% Importação', ascending=False).reset_index(drop=True))

,Setores,% Importação,% Impostos,% Valor Adicionado
0,Fabricação de outros equipamentos de transport...,0.403829,0.0734221,0.522749
1,"Fabricação de equipamentos de informática, pro...",0.391991,0.118597,0.489412
2,Fabricação de químicos orgânicos e inorgânicos...,0.388002,0.108735,0.503263
3,"Fabricação de defensivos, desinfestantes, tint...",0.321716,0.102775,0.575509
4,Fabricação de produtos de borracha e de materi...,0.263568,0.106155,0.630278
5,Metalurgia de metais não-ferrosos e a fundição...,0.256784,0.0908236,0.652393
6,Refino de petróleo e coquerias,0.253931,0.18791,0.558159
7,"Fabricação de automóveis, caminhões e ônibus, ...",0.249646,0.118429,0.631924
8,Fabricação de máquinas e equipamentos elétricos,0.239453,0.101371,0.659176
9,"Manutenção, reparação e instalação de máquinas...",0.23825,0.083035,0.678715


In [ ]:
df_precos['% Impostos'].mean()

0.07887838309877657

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df_precos.sort_values(by='% Impostos', ascending=False).reset_index(drop=True))

,Setores,% Importação,% Impostos,% Valor Adicionado
0,Refino de petróleo e coquerias,0.253931,0.18791,0.558159
1,Transporte aéreo,0.226044,0.161688,0.612268
2,"Energia elétrica, gás natural e outras utilidades",0.0953234,0.131594,0.773082
3,Fabricação de bebidas,0.114439,0.119046,0.766515
4,"Fabricação de equipamentos de informática, pro...",0.391991,0.118597,0.489412
5,"Fabricação de automóveis, caminhões e ônibus, ...",0.249646,0.118429,0.631924
6,"Fabricação de produtos de limpeza, cosméticos/...",0.225273,0.114505,0.660222
7,Fabricação de produtos de minerais não-metálicos,0.157987,0.109376,0.732637
8,Fabricação de calçados e de artefatos de couro,0.135335,0.108842,0.755823
9,Fabricação de químicos orgânicos e inorgânicos...,0.388002,0.108735,0.503263


In [ ]:
df_precos['% Valor Adicionado'].mean()

0.7856775943821063

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df_precos.sort_values(by='% Valor Adicionado', ascending=False).reset_index(drop=True))

,Setores,% Importação,% Impostos,% Valor Adicionado
0,Serviços domésticos,0,0,1
1,Atividades imobiliárias,0.00848222,0.0112707,0.980247
2,Educação pública,0.0241696,0.0205879,0.955242
3,"Atividades de vigilância, segurança e investig...",0.0281004,0.025346,0.946554
4,"Administração pública, defesa e seguridade social",0.0341803,0.0375573,0.928262
5,"Atividades jurídicas, contábeis, consultoria e...",0.0361478,0.0361363,0.927716
6,Produção florestal; pesca e aquicultura,0.0387735,0.0360848,0.925142
7,Educação privada,0.0432919,0.0342446,0.922464
8,"Intermediação financeira, seguros e previdênci...",0.0328494,0.0449223,0.922228
9,Outras atividades administrativas e serviços c...,0.0489306,0.0476584,0.903411


##### Multiplicadores produção

In [ ]:
mMultiplicadores = pd.DataFrame(mMultiplicadores)
mMultiplicadores.sort_values(by='Total', ascending=False)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(mMultiplicadores.sort_values(by='Total', ascending=False).reset_index(drop=True))

,Setores,Total,Simples,Induzido,Direto,Indireto
0,"Abate e produtos de carne, inclusive os produt...",3.992919,2.497733,1.495186,0.776931,1.720802
1,Fabricação e refino de açúcar,3.962771,2.390283,1.572489,0.790644,1.599638
2,Serviços domésticos,3.862777,1.000000,2.862777,0.000000,1.000000
3,Educação pública,3.745857,1.225757,2.520100,0.134944,1.090813
4,Fabricação de biocombustíveis,3.718053,2.334341,1.383712,0.731969,1.602372
5,Saúde pública,3.701542,1.467727,2.233815,0.274497,1.193230
6,Outros produtos alimentares,3.663945,2.312983,1.350961,0.709291,1.603693
7,Educação privada,3.663692,1.387922,2.275771,0.238698,1.149224
8,"Atividades de vigilância, segurança e investig...",3.622674,1.252756,2.369918,0.155040,1.097715
9,"Pecuária, inclusive o apoio à pecuária",3.612762,1.885100,1.727662,0.450258,1.434842


In [ ]:
mMultiplicadores[mMultiplicadores['Setores'] == 'Fabricação de celulose, papel e produtos de papel'].reset_index(drop=True)

,Setores,Total,Simples,Induzido,Direto,Indireto
0,"Fabricação de celulose, papel e produtos de papel",3.199667,2.072068,1.127599,0.579061,1.493008


##### Multiplicadores emprego

In [ ]:
TabelaMultiplicadoresEmprego = pd.DataFrame(TabelaMultiplicadoresEmprego)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(TabelaMultiplicadoresEmprego.sort_values(by='Total', ascending=False).reset_index(drop=True))

,Setores,Total,Simples,Induzido,Direto,Indireto
0,Serviços domésticos,97.350648,88.157795,9.192853,88.157795,0.000000
1,"Pecuária, inclusive o apoio à pecuária",51.560821,46.013013,5.547808,38.769181,7.243832
2,Confecção de artefatos do vestuário e acessórios,39.120462,33.591650,5.528812,25.237626,8.354024
3,Organizações associativas e outros serviços pe...,37.977218,31.959551,6.017667,26.191451,5.768100
4,"Atividades artísticas, criativas e de espetáculos",37.693440,31.753680,5.939760,27.816702,3.936978
5,Alimentação,32.624180,27.070052,5.554128,20.301324,6.768728
6,Educação privada,30.085841,22.777964,7.307878,20.069208,2.708756
7,Produção florestal; pesca e aquicultura,29.199108,24.569168,4.629940,21.153842,3.415326
8,"Abate e produtos de carne, inclusive os produt...",28.541748,23.740458,4.801290,2.616916,21.123542
9,"Atividades de vigilância, segurança e investig...",27.550479,19.940278,7.610201,18.143517,1.796761


In [ ]:
TabelaMultiplicadoresEmprego[TabelaMultiplicadoresEmprego['Setores'] == 'Fabricação de celulose, papel e produtos de papel'].reset_index(drop=True)

,Setores,Total,Simples,Induzido,Direto,Indireto
0,"Fabricação de celulose, papel e produtos de papel",12.029134,8.408228,3.620907,1.841115,6.567113


##### Multiplicadores PIB

In [ ]:
TabelaMultiplicadoresVA = pd.DataFrame(TabelaMultiplicadoresVA)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(TabelaMultiplicadoresVA.sort_values(by='Indireto', ascending=False).reset_index(drop=True))

,Setores,Total,Simples,Induzido,Direto,Indireto
0,"Abate e produtos de carne, inclusive os produt...",1.046759,0.787751,0.259008,0.156738,0.631013
1,Fabricação e refino de açúcar,1.057296,0.784897,0.272399,0.163398,0.621499
2,Fabricação de biocombustíveis,1.022287,0.782589,0.239698,0.210919,0.571670
3,Outros produtos alimentares,1.000066,0.766041,0.234025,0.203199,0.562843
4,Fabricação de produtos do fumo,1.011364,0.783400,0.227964,0.228240,0.555159
5,"Fabricação de automóveis, caminhões e ônibus, ...",0.842588,0.631924,0.210663,0.123529,0.508395
6,Fabricação de bebidas,0.981224,0.766515,0.214710,0.281116,0.485398
7,"Outras atividades profissionais, científicas e...",1.112333,0.850199,0.262134,0.370292,0.479907
8,"Produção de ferro-gusa/ferroligas, siderurgia ...",0.863533,0.696235,0.167298,0.227748,0.468487
9,Refino de petróleo e coquerias,0.674958,0.558159,0.116799,0.094605,0.463553


In [ ]:
TabelaMultiplicadoresVA[TabelaMultiplicadoresVA['Setores'] == 'Fabricação de celulose, papel e produtos de papel'].reset_index(drop=True)

,Setores,Total,Simples,Induzido,Direto,Indireto
0,"Fabricação de celulose, papel e produtos de papel",0.931045,0.735713,0.195332,0.289977,0.445737


##### Multiplicadores renda

In [ ]:
TabelaMultiplicadoresRenda = pd.DataFrame(TabelaMultiplicadoresRenda)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(TabelaMultiplicadoresRenda.sort_values(by='Simples', ascending=False).reset_index(drop=True))

,Setores,Total,Simples,Induzido,Direto,Indireto
0,Serviços domésticos,1.204564,1.000000,0.204564,1.000000,0.000000
1,Educação pública,1.044897,0.864819,0.180078,0.803458,0.061362
2,"Atividades de vigilância, segurança e investig...",0.949680,0.780333,0.169346,0.721041,0.059292
3,Saúde pública,0.906522,0.746901,0.159621,0.629945,0.116956
4,Educação privada,0.879579,0.716960,0.162619,0.626986,0.089974
5,"Administração pública, defesa e seguridade social",0.869550,0.716720,0.152829,0.615656,0.101065
6,Outras atividades administrativas e serviços c...,0.684995,0.557215,0.127779,0.452416,0.104800
7,Edição e edição integrada à impressão,0.672853,0.550172,0.122681,0.379713,0.170460
8,Alojamento,0.674938,0.544402,0.130536,0.406917,0.137485
9,"Atividades de televisão, rádio, cinema e grav...",0.602647,0.485739,0.116909,0.266171,0.219568


In [ ]:
TabelaMultiplicadoresRenda[TabelaMultiplicadoresRenda['Setores'] == 'Fabricação de celulose, papel e produtos de papel'].reset_index(drop=True)

,Setores,Total,Simples,Induzido,Direto,Indireto
0,"Fabricação de celulose, papel e produtos de papel",0.403356,0.322782,0.080574,0.127999,0.194783


##### Multiplicadores impostos

In [ ]:
TabelaMultiplicadoresTributos = pd.DataFrame(TabelaMultiplicadoresTributos)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(TabelaMultiplicadoresTributos.sort_values(by='Total', ascending=False).reset_index(drop=True))

,Setores,Total,Simples,Induzido,Direto,Indireto
0,Refino de petróleo e coquerias,0.197428,0.187910,0.009518,0.109962,0.077948
1,Transporte aéreo,0.177350,0.161688,0.015662,0.109551,0.052136
2,"Energia elétrica, gás natural e outras utilidades",0.141965,0.131594,0.010371,0.075767,0.055827
3,Fabricação de bebidas,0.136544,0.119046,0.017498,0.069144,0.049902
4,"Fabricação de automóveis, caminhões e ônibus, ...",0.135597,0.118429,0.017168,0.062427,0.056003
5,"Fabricação de produtos de limpeza, cosméticos/...",0.131523,0.114505,0.017018,0.063735,0.050770
6,"Fabricação de equipamentos de informática, pro...",0.130935,0.118597,0.012338,0.078698,0.039899
7,Fabricação de calçados e de artefatos de couro,0.129937,0.108842,0.021095,0.065086,0.043756
8,Fabricação de produtos de minerais não-metálicos,0.129362,0.109376,0.019986,0.057212,0.052163
9,"Abate e produtos de carne, inclusive os produt...",0.128554,0.107447,0.021108,0.044704,0.062743


In [ ]:
TabelaMultiplicadoresTributos[TabelaMultiplicadoresTributos['Setores'] == 'Fabricação de celulose, papel e produtos de papel'].reset_index(drop=True)

,Setores,Total,Simples,Induzido,Direto,Indireto
0,"Fabricação de celulose, papel e produtos de papel",0.110186,0.094268,0.015918,0.046514,0.047754


##### Choques multiplicadores

In [ ]:
vSetores[16]

'Fabricação de celulose, papel e produtos de papel'

In [ ]:
mY[16,0]

33268.78930471119

In [ ]:
### Aumento de 5,41% nas exportações do setor de celulose e papel

mAux = np.zeros((nSetores,1))

nChoque = 0.0541
nDeltaY = (nChoque * mY[16,0]).reshape((1,1))

mAux[16,0] = nDeltaY

mDeltaX = np.dot(mLeontief, mAux)

In [ ]:
nDeltaVBP = np.sum(mDeltaX)
nDeltaVBPperc = nDeltaVBP / np.sum(mVA[12,:])

vRelacaoVA = (mVA[0,:] / mVA[12,:]).reshape((68,1))
vDeltaVA = vRelacaoVA * mDeltaX
nDeltaVA = np.sum(vDeltaVA)
nDeltaVAperc = nDeltaVA / np.sum(mVA[0])

vRelacaoRemuneracoes = (mVA[1,:] / mVA[12,:]).reshape((68,1))
vDeltaRemuneracoes = vRelacaoRemuneracoes * mDeltaX
nDeltaRemuneracoes = np.sum(vDeltaRemuneracoes)
nDeltaRemuneracoesperc = nDeltaRemuneracoes / np.sum(mVA[1])

vRelacaoOcupacoes = (mVA[13,:] / mVA[12,:]).reshape((68,1))
vDeltaOcupacoes = vRelacaoOcupacoes * mDeltaX
nDeltaOcupacoes= np.sum(vDeltaOcupacoes)
nDeltaOcupacoesperc = nDeltaOcupacoes / np.sum(mVA[13])

vRelacaoImpostos = (mMIP[70,:68] / mVA[12,:]).reshape((68,1))
vDeltaImpostos = vRelacaoImpostos * mDeltaX
nDeltaImpostos = np.sum(vDeltaImpostos)
nDeltaImpostosperc = nDeltaImpostos / np.sum(mVA[13])